Este é o Notebook oficial do grupo 4, aqui teremos:
- um resumo da análise de dados;
- os códigos responsáveis pelo processamento (tratamento) dos dados da tabela;
- o modelo preditivo;

Em todos os códigos, há comentários suficientes para explicar o objetivo e funcionamento da função.

# Importações e instalações

In [ ]:
!pip install dataprep
!pip install moment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

# Bibliotecas 
import pandas as pd
import datetime as dt
from datetime import date
import numpy as np
from sklearn import preprocessing
import moment

## Bibliotecas para os gráficos
import matplotlib.pyplot as plt #geração de gráficos
import seaborn as sns #template de gráficos
import plotly.express as px #geração de gráficos dinâmicos
import plotly.offline as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt

### Modelos preditivos:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier # Decision Tree model
from sklearn.tree import plot_tree
from sklearn.neighbors import KNeighborsClassifier # KNN model
from sklearn import svm # SVM model
from sklearn.naive_bayes import GaussianNB # naive bayes model
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import metrics


#### Métricas de avaliação
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import ConfusionMatrixDisplay

py.init_notebook_mode(connected=True) 

In [ ]:
# Conectando ao google drive
from google.colab import drive
drive.mount('/content/drive')
content_link = '/content/drive/Shareddrives/G444 Drive/docs everymind/Base Colaboradores Everymind_Inteli_2020 a 2022vModelo Preditivo.xlsx'

Mounted at /content/drive


In [ ]:
# Importando as planilhas
df1 = pd.read_excel(content_link)
df2 = pd.read_excel(content_link, sheet_name = 'Reconhecimento')
df3 = pd.read_excel(content_link, sheet_name = 'Ambiente de Trabalho 27.07')

# Análise de Dados - Resumo

# Tratamento de Dados

Resumo da seção:
- Modificação de dados 
- Categorização
- Derivação

## Modificação de dados

- Eliminiação de espaços
- Tipificação de dados (como datas em string para datetime)

In [ ]:
# Aqui percorremos todas as linhas das colunas e fazer a alteração de onde possui espaço em branco e substituir para vazio 
# (percorremos apenas linhas em string, por isso o argumento 'object' e o atributo str)
for i in df1.select_dtypes(include = 'object').columns.drop('Dt Admissao'):
  df1[i] = df1[i].str.replace(' ','')

In [ ]:
for i in df2.select_dtypes(include = 'object'):
  df2[i] = df2[i].str.replace(' ','')

In [ ]:
# Aqui substituímos o texto de "PessoaColaboradora" de todas as linhas da coluna 'Nome Completo' por vazio 
# (isso tem como objetivo obter apenas o número do colaborador)
for i in range(0, len(df1['Nome Completo'])):
  df1['Nome Completo'][i] = df1['Nome Completo'][i].replace('PessoaColaboradora', '')

In [ ]:
df1 = df1.rename(columns={'Matrícula': 'Matricula'})
df1['Estagnação'] = 0

In [ ]:
for i in range(0, len(df2['Codinome'])):
  df2['Codinome'][i] = df2['Codinome'][i].replace('PessoaColaboradora', '')

In [ ]:
#Substituindo os valores das colunas da tabela 2 por valores numéricos
for column in df2.drop(['Data de Admissão',	'Data Vigência', 'Novo Salario'], axis=1):
  df2[column] = preprocessing.LabelEncoder().fit(df2[column]).transform(df2[column])

## Categorização
Aqui categorizamos os dados para numérico para tratar melhor com o modelo preditivo (como categorizar Tipo de Saída dos funcionários)

In [ ]:
# Categorização do gênero dos funcionários
# 0 significa Masculino
# 1 significa Feminino
df1['Genero_Numerico'] = (df1['Genero']
                          .replace('Masculino', 0)
                          .replace('Feminino', 1))

In [ ]:
# Categorização do tipo de saída dos funcionários
# 0 significa ativo
# 1 significa rescizão de contrato por pedido de demissão
# 2 significa rescisão de contrato por demissão
# 3 significa demissão
# 4 significa pedido de demissão
df1['Tipo_Saida_Numerico'] = (df1['Tipo Saida']
                              .fillna(0)
                              .replace('RescisaoContratoExp-Dispensa', 1)
                              .replace('RescisaoContratoExp-Pedido', 2)
                              .replace('DispensasemJustaCausa', 3)
                              .replace('PedidodeDemissão', 4))

In [ ]:
# Categorização dos Estados para futuros cruzamentos de dados
df1['Estado_Numerico'] = preprocessing.LabelEncoder().fit_transform(df1['Estado'])

In [ ]:
# Categorização das Regiões (agrupando os Estados)
# '1' para Norte
# '2' para Nordeste
# '3' para Centro-Oeste
# '4' para Sudeste
# '5' para Sul
df1['Regiao_Numerico'] = (df1['Estado']
                       # Norte
                       .replace('AM', 1)
                       .replace('RR', 1)
                       .replace('AC' , 1)
                       .replace('RO', 1)
                       .replace('AP', 1)
                       .replace('PA', 1)
                       .replace('TO', 1)
                       # Nordeste
                       .replace('MA', 2)
                       .replace('CE', 2)
                       .replace('PI', 1)
                       .replace('RN', 1)
                       .replace('PB', 1)
                       .replace('PE', 1)
                       .replace('AL', 1)
                       .replace('SE', 1)
                       .replace('BA', 1)
                       # Centro
                       .replace('MS', 1)
                       .replace('GO', 1)
                       .replace('DF', 1)
                       .replace('MT', 3)
                       # Sudeste
                       .replace('MG', 4)
                       .replace('ES', 4)
                       .replace('RJ', 4)
                       .replace('SP', 4)
                       # Sul
                       .replace('PR', 4)
                       .replace('SC', 4)
                       .replace('RS', 4)
                       )

In [ ]:
# Categorização dos Cargos para futuros cruzamentos de dados
df1['Cargo_Numerico'] = preprocessing.LabelEncoder().fit_transform(df1['Cargo'])

In [ ]:
#


In [ ]:
# Categorização da situação dos funcionários
# 0 significa que o funcionário foi desligado
# 1 significa que o funcionário está ativo
df2['Situação_Numerico'] = (df2['Situação']
                          .replace('Desligado', 0)
                          .replace('Ativo', 1)
                          .replace('Afastado',1))

In [ ]:
# Exibir lista de ocorrências das variáveis categóricas
# Qtd de ocorrencia de cada uma das categorias
print(df2.Situação.unique())
df2.Situação.value_counts()

## Derivação
Aqui criamos algumas variáveis a mais a partir dos dados que temos para melhorar o modelo preditivo (como criar a idade a partir da data de nascimento)

### Status

In [ ]:
# Criação da coluna status, onde 
# "0" é "desativo" 
# "1" é "ativo"
df1['Status'] = (df1['Tipo Saida']
                              .fillna(0)
                              .replace('RescisaoContratoExp-Dispensa', 1)
                              .replace('RescisaoContratoExp-Pedido', 1)
                              .replace('DispensasemJustaCausa', 1)
                              .replace('PedidodeDemissão', 1))

In [ ]:
df1['Tipo_Saida_Numerico'].value_counts()

In [ ]:
df1['Status'].value_counts()

### Média Salarial 

In [ ]:
df1['Media_Salarial'] = -1

for i in range(0, len(df1)):
  Cargo_da_pessoa = df1['Cargo_Numerico'][i]
  df1['Media_Salarial'][i] = df1.query(f'`Cargo_Numerico` == {Cargo_da_pessoa}').mean()['Salario Mês']

In [ ]:
df1['Salario_Comparado'] = 0

for item in range(0,len(df1)):
    if df1['Media_Salarial'][item] > df1['Salario Mês'][item]:
        df1['Salario_Comparado'][item] = 1  
df1

In [ ]:
df1['salario_comparado'] = 0

for item in range(0,len(df1)):
    if df1['Media_Salarial'][item] > df1['Salario Mês'][item]:
        df1['salario_comparado'][item] = 1  
df1

### Jornada de trabalho

In [ ]:
#Função pega a data de admissão do colaborador e a data do seu desligamento, e encontra o perído entre elas.
df1['Tempo de Trabalho'] = (pd.to_datetime(df1['Dt Saida']) - pd.to_datetime(df1['Dt Admissao'])).replace(np.NaN, date.today())

### Idade

In [ ]:
#Ele pega a data de hoje e subtrai da data de nascimento, retornando a idade, np.timedelta64, transforma o retorno da data de dias para ano.
df1['Idade'] = ((pd.to_datetime('today')-pd.to_datetime(df1['Dt Nascimento']))/ np.timedelta64(1, 'Y')).astype(int)
df1['Idade']

###Faixa Etária

In [ ]:
df1['faixa_etaria'] = 0
for i in range(0, len(df1)):
    if df1['Idade'][i] >= 18 and  df1['Idade'][i] <= 21:
        df1['faixa_etaria'][i] = '0'
    elif df1['Idade'][i] >= 22 and  df1['Idade'][i] <= 25:
        df1['faixa_etaria'][i] = '1'
    elif df1['Idade'][i] >= 26 and  df1['Idade'][i] <= 29:
        df1['faixa_etaria'][i] = '2'
    elif df1['Idade'][i] >= 30 and  df1['Idade'][i] <= 33:
        df1['faixa_etaria'][i] = '3'
    elif df1['Idade'][i] >= 34 and  df1['Idade'][i] <= 37:
        df1['faixa_etaria'][i] = '4'
    elif df1['Idade'][i] >= 38 and  df1['Idade'][i] <= 41:
        df1['faixa_etaria'][i] = '5'
    elif df1['Idade'][i] >= 42 and  df1['Idade'][i] <= 45:
        df1['faixa_etaria'][i] = '6'
    elif df1['Idade'][i] >= 46 and  df1['Idade'][i] <= 49:
        df1['faixa_etaria'][i] = '7'
    elif df1['Idade'][i] >= 50 and  df1['Idade'][i] <= 65:
        df1['faixa_etaria'][i] = '8'


###Período de estagnação de reconhecimento

O período de estagnação está em dias, e é o período de tempo entre o reconhecimento e o dia de hoje.

In [ ]:
df2['Estagnação'] = ((pd.to_datetime('today')-pd.to_datetime(df2['Data Vigência']))/ np.timedelta64(1, 'D')).astype(int)

### Estagnação na df1

In [ ]:
# def get_estagnacao(x): #define a funcao
#   df_funcionario = df2[df2['Matricula']==x['Matricula']] #cria um dataframe com apenas os dados que estão nas duas tabelas ao mesmo tempo
#   return df_funcionario['Estagnação'].min() #devolve, desse dataframe, apenas o menor valor de estagnação
# df1['Estagnação'] = df1.apply(get_estagnacao, axis=1)

In [ ]:
contador = 0
for matricula in df1['Matricula']:
  if matricula in df2['Matricula'].values:
    query = df2.query(f'Matricula == {matricula}')
    if query['Matricula'].count() > 1:
      df1['Estagnação'][contador] = np.array(query['Estagnação']).min()
    else:
      df1['Estagnação'][contador] = query['Estagnação']
  else:
    if df1.query(f'Matricula == {matricula}')['Status'][contador] == 1:
      df1['Estagnação'][contador] = round(((pd.to_datetime(df1['Dt Saida'][contador])-pd.to_datetime(df1['Dt Admissao'][contador]))/np.timedelta64(1, 'D')))
    else:
      df1['Estagnação'][contador] = round(((pd.to_datetime('today')-pd.to_datetime(df1['Dt Admissao'][contador]))/np.timedelta64(1, 'D')))

  contador += 1

## Seleção de dados

### Quantidades de reconhecimento

Em ordem: 
- Pessoas que saíram
- Pessoas que não saíram
- Pessoas que foram reconhecidas de alguma forma
- Pessoas que não foram reconhecidas
- Pessoas que saíram e foram reconhecidas
- Pessoas que saíram e não foram reconhecidas 

In [ ]:
# Pessoas que saíram: ✔
pessoas_desativadas = df1.dropna(subset=['Dt Saida'])['Nome Completo'].unique()
qtd_desativados = len(pessoas_desativadas)
qtd_desativados

In [ ]:
# Pessoas que não saíram: ✔
pessoas_ativas = df1.query('`Dt Saida` == ""')['Nome Completo'].unique()
len(pessoas_ativas)

In [ ]:
# Pessoas que foram reconhecidas de alguma forma: ✔
pessoas_reconhecidas = df2['Codinome'].unique()
len(pessoas_reconhecidas)

In [ ]:
# Pessoas que não foram reconhecidas: ✔
lista_pessoas = df1['Nome Completo'].unique()
pessoas_sem_reconhecimento = []
for i in range(0, len(lista_pessoas)):
  if lista_pessoas[i] in pessoas_reconhecidas:
    continue
  else:
    pessoas_sem_reconhecimento.append(lista_pessoas[i])
len(pessoas_sem_reconhecimento)

In [ ]:
  # Pessoas que foram reconhecidas e saíram ✔
pessoas_desativadas_reconhecidas = df2.query('`Situação` == "Desligado"')['Codinome'].unique()
len(pessoas_desativadas_reconhecidas)

In [ ]:
# Pessoas que não foram reconhecidas e saíram:
pessoas_desativadas_sem_reconhecimento = []
for pessoa in pessoas_desativadas:
  if pessoa in pessoas_reconhecidas:
    continue
  else:
    pessoas_desativadas_sem_reconhecimento.append(pessoa)
print(len(pessoas_desativadas_sem_reconhecimento))

###Reconhecimento por colaborador


In [ ]:
num_prom = df2['Codinome'].value_counts()

In [ ]:
num_prom

In [ ]:
df2['Reconhecimento Num'] = df2['Codinome'].replace(num_prom)

In [ ]:
df2

### Gênero

In [ ]:
gen_total = df1.groupby(['Genero_Numerico']).count()
gen_total

In [ ]:
gen_total_masc = gen_total.iloc[0, 1]
gen_total_masc

In [ ]:
gen_total_fem = gen_total.iloc[1, 1]
gen_total_fem

In [ ]:
gen_masc_left = gen_total.iloc[0, 3]
gen_masc_left

In [ ]:
gen_fem_left = gen_total.iloc[1, 3]
gen_fem_left

In [ ]:
gen_fem_prop = (gen_fem_left/gen_total_fem)*100
gen_fem_prop

In [ ]:
gen_masc_prop = (gen_masc_left/gen_total_masc)*100
gen_masc_prop

# Saída por estado 

In [ ]:
df1['estadoSP'] = 0

for item in range(0,len(df1)):
    if df1['Estado_Numerico'][item] == 16:
        df1['estadoSP'][item] = 1  
df1

In [ ]:
Estado_total = df1.groupby(['estadoSP']).count()
Estado_total

In [ ]:
total_foraSP = Estado_total.iloc[0, 1]
total_foraSP

In [ ]:
total_emSP = Estado_total.iloc[1, 1]
total_emSP

In [ ]:
total_foraSP_off = Estado_total.iloc[0, 3]
total_foraSP_off

In [ ]:
total_emSP_off = Estado_total.iloc[1, 3]
total_emSP_off

In [ ]:
emSP_prop = (total_emSP_off/total_emSP)*100
emSP_prop

In [ ]:
foraSP_prop = (total_foraSP_off/total_foraSP)*100
foraSP_prop

# Gráficos

### Porcentagem de saídas em gênero 

In [ ]:
gen_x = ['mulheres', 'homens']
gen_y = [gen_fem_prop, gen_masc_prop]

plt.bar(gen_x, gen_y)

plt.xlabel('Gênero')
plt.ylabel('Desligamentos (em %)')
plt.title('Desligamentos x Gênero (proporcionalmente)')

# Porcentagem de saída por estado

In [ ]:
est_x = ['São Paulo', 'Fora de São Paulo']
est_y = [emSP_prop,foraSP_prop]

plt.bar(est_x, est_y)

plt.xlabel('Local')
plt.ylabel('Desligamentos (em %)')
plt.title('Desligamentos x cidade (proporcionalmente)')

# Modelagem

## Seleção de dados para a modelagem:

In [ ]:
# Primeiro separamos as pessoas ativas e desativas em dataframes diferentes
df_ativas = df1.query('Status == 1')
df_desativas = df1.query('Status == 0')
df_ativas

In [ ]:
# Dois terços dos ativos e desativos será treino:
len(df_ativas) # isso retorna 284 e dois terços disso será treino
ativas_treino = df_ativas.head(189) 
ativas_teste = df_ativas.tail(95)

len(df_desativas) # isso retorna 191, dois terços disso será treino
desativas_treino = df_desativas.head(127)
desativas_teste = df_desativas.tail(64)

In [ ]:
# Com, isso, podemos juntar os dataframes de treino e teste:
df_treino = np.array(pd.merge(ativas_treino, desativas_treino, how = 'outer'))
df_teste = np.array(pd.merge(ativas_teste, desativas_teste, how = 'outer'))

In [ ]:
df1['Status'].value_counts()

## Modelo SVM (support-vector machine)

In [ ]:
#Divisão de dados para treino '30%'
x_SVM = df1[['Idade', 'Cargo_Numerico', 'Regiao_Numerico', 'Salario_Comparado','Estagnação']]
y_SVM = df1['Status']
x_SVM_train, x_SVM_test, y_SVM_train, y_SVM_test = train_test_split(x_SVM,y_SVM,test_size=0.3, random_state = 0)

In [ ]:
SVM_model = svm.SVC(gamma='auto')
SVM_model.fit(x_SVM_train, y_SVM_train)

y_SVM_pred = SVM_model.predict(x_SVM_test)
SVM_score = accuracy_score(y_SVM_pred, y_SVM_test)
print(y_SVM_pred)
print("Acurácia: ", SVM_score)

In [ ]:
cm = confusion_matrix(y_SVM_test, y_SVM_pred)
disp = ConfusionMatrixDisplay(confusion_matrix = cm)
disp.plot()
plt.show

## Modelo KNN

In [ ]:
#split dataset
x= df1[['Cargo_Numerico', 'Idade']]
y= df1['Status']

x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size = 0.4, 
                                                    random_state = 0)

print(x_train.shape , x_test.shape, y_test.shape, y_train.shape)  

In [ ]:
#feature scaling
sc_x = StandardScaler()
x_train = sc_x.fit_transform(x_train)
x_test = sc_x.transform(x_test)

In [ ]:
#para definir o k
import math
math.sqrt(len(y_test))

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=13, metric='euclidean')
neigh.fit(x_train, y_train.squeeze())

In [ ]:
print('Acuracia (treino): ', neigh.score( x_train, y_train ))
print('Acuracia (teste): ', neigh.score( x_test, y_test ))

In [ ]:
y_pred = neigh.predict(x_test)
y_pred

In [ ]:
#avaliando o modelo
cm = confusion_matrix(y_test, y_pred)
print (cm)


In [ ]:
plot_confusion_matrix(neigh, x_test, y_test, cmap= "Blues")

In [ ]:
print(f1_score(y_test, y_pred))

In [ ]:
print(accuracy_score(y_test, y_pred))

In [ ]:
df1

##Modelo Árvore de decisão

In [ ]:
df1.columns

In [ ]:
x = df1[['faixa_etaria', 'Salario_Comparado', 'estadoSP', 'Estagnação']]
y = df1['Status']

print(x)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

arv = DecisionTreeClassifier(criterion='entropy',random_state=42)
arv.fit(x_train, y_train)

p = arv.predict(x_test)

fig, ax = plt.subplots(figsize=(100, 100))
_ = plot_tree(arv, feature_names=x.columns)

In [ ]:
plot_confusion_matrix(arv, x_test, y_test, cmap='Blues', values_format='.0f')

Acurácia usando o status

In [ ]:
score = accuracy_score(p, y_test)
score

## Modelo de Naive Bayes


In [ ]:
#Divisão de dados para treino '30%'
x = df1[['Idade', 'Cargo_Numerico', 'Regiao_Numerico','estadoSP', 'salario_comparado']]
y = df1['Status']
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3, random_state = 0)
print(y)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics 

model = GaussianNB()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
score = accuracy_score(y_pred, y_test)
print('Accuracy:', score)

In [ ]:
y_true = y_test

In [ ]:
cm = confusion_matrix(y_true, y_pred)
cm

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix = cm)

disp.plot()
plt.show()

##Modelo de Regressão Logística

In [ ]:
# Dividindo x e y
x = df1[['faixa_etaria', 'Cargo_Numerico', 'Regiao_Numerico', 'Estado_Numerico', 'Salario Mês']]
# PS: NUNCA ESQUEÇA DE TIRAR A RESPOSTA DO CONJUNTO DE CARACTERÍSTICAS!!!
y = df1['Status']
# Dividindo dados para treino e dados para teste
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size = 0.3,
                                                    random_state = 42)
# Treinando o modelo
model = LogisticRegression().fit(x_train, y_train)
# Fazendo as predições
y_pred = model.predict(x_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
y_test.value_counts()

In [ ]:
confusion_matrix(y_test, y_pred)


In [ ]:
_ = plot_confusion_matrix(model, x_test, y_test, cmap = 'Blues', values_format='.0f')

In [ ]:
print(classification_report(y_test, y_pred))

## Redes Neurais

In [ ]:

class NeuralNetwork():
    
    def __init__(self):
        # Seed the random number generator
        np.random.seed(1)

        # Set synaptic weights to a 3x1 matrix,
        # with values from -1 to 1 and mean 0
        self.synaptic_weights = 2 * np.random.random((3, 1)) - 1

    def sigmoid(self, x):
        """
        Takes in weighted sum of the inputs and normalizes
        them through between 0 and 1 through a sigmoid function
        """
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        """
        The derivative of the sigmoid function used to
        calculate necessary weight adjustments
        """
        return x * (1 - x)

    def train(self, training_inputs, training_outputs, training_iterations):
        """
        We train the model through trial and error, adjusting the
        synaptic weights each time to get a better result
        """
        for iteration in range(training_iterations):
            # Pass training set through the neural network
            output = self.think(training_inputs)

            # Calculate the error rate
            error = training_outputs - output

            # Multiply error by input and gradient of the sigmoid function
            # Less confident weights are adjusted more through the nature of the function
            adjustments = np.dot(training_inputs.T, error * self.sigmoid_derivative(output))

            # Adjust synaptic weights
            self.synaptic_weights += adjustments

    def think(self, inputs):
        """
        Pass inputs through the neural network to get output
        """
        
        inputs = inputs.astype(float(1), float(0))
        output = self.sigmoid(np.dot(inputs, self.synaptic_weights))
        return output


if __name__ == "__main__":

    # Initialize the single neuron neural network
    neural_network = NeuralNetwork()

    print("Random starting synaptic weights: ")
    print(neural_network.synaptic_weights)

    # The training set, with 4 examples consisting of 3
    # input values and 1 output value
    training_inputs = df_treino
    training_outputs = df_treino

    # Train the neural network
    neural_network.train(training_inputs, training_outputs, 10000)

    print("Synaptic weights after training: ")
    print(neural_network.synaptic_weights)

    A = str(input("Input 1: "))
    B = str(input("Input 2: "))
    C = str(input("Input 3: "))
    
    print("New situation: input data = ", A, B, C)
    print("Output data: ")
    print(neural_network.think(np.array([A, B, C])))

# Testes

In [ ]:
from dataprep.eda import create_report

In [ ]:
create_report(df1).show()

#